In [1]:
%matplotlib inline

In [24]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
import copy
import metpy
import metpy.calc
import os
import time

In [4]:
# we will set the seed this time to ensure fairness
keras.utils.set_random_seed(15)

In [5]:
al_asos = pd.read_csv("../Week_6/al_asos_jul_23_v2.csv")
#us_asos = pd.read_csv("../../../../combined_asos_2023.csv")
#us_asos = pd.read_csv("/share/share/")

In [42]:
def make_generator_model():
    input1 = keras.layers.Input(shape=(100, ))

    dense1 = keras.layers.Dense(128, input_dim=2, activation=None)(input1)
    lerelu_layer = keras.layers.LeakyReLU(alpha=0.3)(dense1) 

    output = keras.layers.Dense(3, activation=keras.activations.relu)(lerelu_layer)
    model_out = keras.models.Model(inputs=[input1], outputs=[output])
    assert model_out.output_shape == (None, 3)
    return model_out



In [43]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_out = generator(noise, training=False)


In [44]:
generated_out

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.47215185, 0.        , 0.        ]], dtype=float32)>

In [45]:
def make_discriminator_model():
    input1 = keras.layers.Input(shape=(3, ))
    input_norm = keras.layers.Normalization(axis=1)(input1)


    dense1 = keras.layers.Dense(128, activation=None)(input_norm)
    lerelu_layer = keras.layers.LeakyReLU(alpha=0.3)(dense1) 
    dropout = keras.layers.Dropout(0.3)(lerelu_layer)

    output = keras.layers.Dense(1, activation=keras.activations.relu)(dropout)
    model_out = keras.models.Model(inputs=[input1], outputs=[output])
    assert model_out.output_shape == (None, 1)
    return model_out


In [46]:
discriminator = make_discriminator_model()
decision = discriminator(generated_out)
print (decision)

tf.Tensor([[0.]], shape=(1, 1), dtype=float32)


In [47]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [48]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss


In [49]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [58]:
generator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.legacy.Adam(1e-4)


In [59]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


In [89]:
EPOCHS = 5
noise_dim = 100
num_examples_to_generate = 16
BATCH_SIZE=256
BUFFER_SIZE=60000

In [96]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(inputs):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(inputs, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


In [97]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)


    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))



In [98]:
al_asos_nona = al_asos.dropna(subset=['tmpf', 'relh', 'dwpf'])

In [99]:
train_dataset = tf.data.Dataset.from_tensor_slices(al_asos_nona[['tmpf', 'relh', 'dwpf']].values).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [101]:
train(train_dataset, EPOCHS)

Time for epoch 1 is 2.050711154937744 sec


KeyboardInterrupt: 